In [ ]:
import numpy as np
import pandas as pd

import os, urllib, time
os.environ["MLFLOW_TRACKING_URI"] = "sqlite:///mlruns/db.sqlite"
import mlflow, git
mlflow_client = mlflow.tracking.MlflowClient()

In [ ]:
def leaders(exp_name, model_name=None):
    exp_id = mlflow.get_experiment_by_name(exp_name).experiment_id
    print("# found exp_id=", exp_id, "for exp_name=", exp_name)
    
    rank_metrics = ['|alpha_tmu|', '|eyelevel_tmu|', '-reward_total']
    print_metrics = ['model_version', 'alpha_tmu', 'alpha_tsigma', 'alpha_tdf', 'eyelevel_tmu', 'eyelevel_tsigma', 'eyelevel_tdf', 'reward_total']

    parent_runs = mlflow_client.search_runs(exp_id) # FIXME filter out child runs
    print("# found ", len(parent_runs), " runs")

    best_metric = dict()
    best_run = dict()
    
    for parent_run in parent_runs:
        #print(f"good one {parent_run}")
        run_name = parent_run.data.tags['mlflow.runName']
        if not run_name.startswith(f"{model_name} "):
            continue
            
        print(f"# processing {run_name}")
            
        child_runs = mlflow_client.search_runs(exp_id, f"tags.mlflow.parentRunId='{parent_run.info.run_id}'")
        #                                           order_by=["metrics.reward_total DESC"])
        print("# found ", len(child_runs), " child_runs")
        for child_run in child_runs:
            if 'alpha_tmu' not in child_run.data.metrics:
                continue

            child_run.data.metrics['|alpha_tmu|'] = np.abs(child_run.data.metrics['alpha_tmu'])
            child_run.data.metrics['|eyelevel_tmu|'] = np.abs(child_run.data.metrics['eyelevel_tmu'])
            child_run.data.metrics['-reward_total'] = - child_run.data.metrics['reward_total']

            for metric in rank_metrics:
                if metric not in best_metric or child_run.data.metrics[metric] < best_metric[metric]:
                    best_metric[metric] = child_run.data.metrics[metric]
                    best_run[metric] = child_run
    metric_values = []
    for whats_best, run in best_run.items():
        best_values = [run.data.metrics[metric] for metric in print_metrics]
        metric_values.append([whats_best] + best_values)

    return pd.DataFrame(metric_values, columns=['whats_best'] + print_metrics)

In [ ]:
leaders("PYB-6J-3S-1A", "eos3d.6j-coarse-aim")

In [ ]:
#leaders(3, "Train 3J for 3M steps", "training")